In [ ]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import random as rd

In [ ]:
def criarDF(nomeTxt):
    lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"', '‘', '’', '"', ']', '’']
    txt = open(nomeTxt, 'r', encoding='utf-8')
    dfA = pd.DataFrame()
    linhas = txt.readlines()
    for linha in linhas:
        vetor = linha.replace("'", '').replace('[', '').replace(']\n', '').split(', ')
        for i in range(len(vetor)):
            for char in lista:
                vetor[i] = vetor[i].replace(char, '')

        dfA = pd.concat([dfA, pd.DataFrame(data=[[vetor[0], vetor[1:]]], columns=[['tag', 'noticias']])], ignore_index=True)
    return dfA

In [ ]:
dfFalso = criarDF('falsasTREINO.txt')
dfReal = criarDF('reaisTREINO.txt')

In [ ]:
dfFalso

,tag,noticias
0,f,"[atenção, título, novos, ja, vem, com, qr, cod..."
1,f,"[não, existia, nas, urnas, este, ano, vai, exi..."
2,f,"[relatório, do, pl, dá, xeque-mate, no, tse, o..."
3,f,"[quero, meu, voto, impresso, rataiada, unida, ..."
4,f,"[gilmar, mendes, reconhece, que, sistema, elei..."
...,...,...
1363,f,"[brasil, que, não, tinha, nem, um, dólar, de, ..."
1364,f,"[o, tse, apresentou, 279, mil, urnas, sem, num..."
1365,f,"[ministra, confirma, que, lula, não, ganhou, a..."
1366,f,"[invadiram, a, sede, da, polícia, federal, em,..."


In [ ]:
def separarTreinoTeste(dfA):
    dfB = pd.DataFrame()
    dfC = pd.DataFrame()
    for i in range(len(dfA)):
        if (rd.random() > 0.8):
            aux = dfA.iloc[i, 1]

            aux1 = '['
            for i in aux:
                aux1 += aux + ', '
            aux1 += ']'

            dfB = pd.concat([dfB, aux1], ignore_index=True)
        else:
            aux = pd.DataFrame(dfA.iloc[i, 1])
            print(aux)
            dfC = pd.concat([dfC, aux], ignore_index=True)

    #dfB = dfB[['tag', 'noticias']].value_counts().reset_index()
    #dfC = dfC[['tag', 'noticias']].value_counts().reset_index()
    #dfB.rename(columns={('tag',): 'tag',('noticias',):'palavras', 'count':'contagem'}, inplace=True)
    #dfC.rename(columns={('tag',): 'tag',('noticias',):'palavras', 'count':'contagem'}, inplace=True)
    return [dfB, dfC]


In [ ]:
[dfFalsoTeste, dfFalsoTreino] = separarTreinoTeste(dfFalso)
[dfRealTeste, dfRealTreino] = separarTreinoTeste(dfReal)

         0       1      2   3    4    5   6     7   8     9
0  atenção  título  novos  ja  vem  com  qr  code  de  lula
    0        1    2      3     4    5    6        7       8     9   ...  \
0  não  existia  nas  urnas  este  ano  vai  existir  quando  você  ...   

        40  41       42 43   44    45      46      47      48        49  
0  negócio  do  confira  o  seu  voto  acabar  demora  alguns  segundos  

[1 rows x 50 columns]
          0   1   2   3           4   5    6  7    8      9          10   11  \
0  relatório  do  pl  dá  xeque-mate  no  tse  o  que  estão  esperando  pra   

      12    13       14        15  
0  tirar  esse  governo  impostor  
      0    1     2         3         4      5         6  7       8  9  10  \
0  quero  meu  voto  impresso  rataiada  unida  roubando  o  brasil  e  o   

           11                    12              13  
0  brasileiro  queromeuvotoimpresso  votoimpressoja  
       0       1          2    3        4          5    6     

KeyboardInterrupt: 

In [ ]:
dfFalsoTeste

,0,1,2,3,4,5,6,7,8,9,...,218,219,220,221,222,223,224,225,226,227
0,o,presidente,da,holanda,confirma,que,houve,fraude,na,eleição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,stf,quer,prender,quem,acredita,em,deus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,olha,a,globo,hoje,de,manhã,reconhecendo,a,vulnerabilidade,da,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ele,informou,para,a,gente,que,quando,foi,confirmar,a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,os,bancos,de,dados,das,eleições,são,outros,alheios,aos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,promover,políticas,de,combate,ao,assédio,moral,e,sexual,nos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,implementar,o,programa,de,conscientização,assistência,e,suporte,à,saúde,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,mega,da,virada,2021,duas,apostas,dividem,prêmio,de,r$,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,lula,da,silva,é,um,político,criminoso,e,agora,ocupa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dfFalsoTeste.rename(columns={'contagem':'aparicoesFalsas'}, inplace=True)
dfFalsoTreino.rename(columns={'contagem':'aparicoesFalsas'}, inplace=True)
dfRealTeste.rename(columns={'contagem':'aparicoesReais'}, inplace=True)
dfRealTreino.rename(columns={'contagem':'aparicoesReais'}, inplace=True)

In [ ]:
import numpy as np
def validar(dfReal, dfFalso, noticia):
    # Junta os dataframes e une as palavras
    dfJunto = dfReal.merge(dfFalso, how='outer', on='palavras')

    # Conta quantas palavras tem em uma e não tem na outra
    nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
    nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

    # Preenche as palavras que não aparecem com 1
    dfJunto.fillna(1, inplace=True)

    # Soma as palavras com 0 aparições com 1 para evitar multiplicação por 0
    dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
    dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

    # Soma o total de aparições
    totalReal = dfJunto['aparicoesReais'].sum()
    totalFalso = dfJunto['aparicoesFalsas'].sum()

    # Balanceia a ordem de grandeza das aparições
    dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

    # Faz o coeficiente de bray courtis
    dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

    # Limpa palavras com coeficiente menor que 0.3 (palavras com aparições equivalentes em ambos casos)
    dfJunto = dfJunto.loc[dfJunto['bc'] > 0.3]

    # Pega o total de aparições após NLP
    tFalsas = dfJunto['aparicoesFalsas'].sum()
    tReais = dfJunto['aparicoesReais'].sum()

    # Define um df de teste
    dfJuntoTeste = dfJunto

    # Faz os calculos das probabilidades
    dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
    dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)

    # Pega o log na base 10
    dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
    dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

    # Limpa a notícia
    noticia = noticia.lower()
    noticia = noticia.split()
    lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&']
    noticiaFormatada = []

    for palavras in noticia:
        for char in lista:
            palavras = palavras.replace(char, '')
        noticiaFormatada.append(palavras)

    # Cria um df com a notícia limpa
    dfNoticia = pd.DataFrame(data = noticiaFormatada, columns=['palavra'])

    # Junta o df da noticia com o das probabilidades
    dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

    # Faz a soma das probabilidades
    iFalsa = dfNoticia['aparicoesFalsas'].sum()
    iReal = dfNoticia['aparicoesReais'].sum()

    # Confere o resultado
    if (iFalsa > iReal):
        return False
    else:
        return True

In [ ]:
for i in range(36):
    [dfFalsoTeste, dfFalsoTreino] = separarTreinoTeste(dfFalso)
    [dfRealTeste, dfRealTreino] = separarTreinoTeste(dfReal)

    dfTeste = dfRealTeste

    resultado = validar(dfFalsoTreino, dfRealTreino, dfTeste)

In [ ]:
teste = validar(dfFalsoTreino, dfRealTreino, )

CÓDIGO ANTIGO

In [ ]:
dfJunto = dfReal.merge(dfFalso, how='outer')

In [ ]:
nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

In [ ]:
dfJunto.fillna(1, inplace=True)

In [ ]:
dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

In [ ]:
totalReal = dfJunto['aparicoesReais'].sum()
totalFalso = dfJunto['aparicoesFalsas'].sum()

In [ ]:
dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

In [ ]:
dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

In [ ]:
dfJunto = dfJunto.loc[dfJunto['bc'] > 0.3]

In [ ]:
tFalsas = dfJunto['aparicoesFalsas'].sum()
tReais = dfJunto['aparicoesReais'].sum()

In [ ]:
dfJunto

,palavras,aparicoesReais,aparicoesFalsas,bc
0,%,1.0,26.093942,0.926183
1,&,1.0,14.403856,0.870162
4,-pe,1.0,3.966279,0.597284
7,0,5.0,0.626255,0.777381
13,0000,1.0,3.966279,0.597284
...,...,...,...,...
19233,😡,1.0,11.272583,0.837035
19234,🙄🙄🙄,1.0,22.336414,0.914297
19235,🚨,1.0,3.966279,0.597284
19236,🤣🤣🤣,1.0,22.336414,0.914297


In [ ]:
dfJuntoTeste = dfJunto

In [ ]:
dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)
dfJuntoTeste

C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\229914731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\229914731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)


,palavras,aparicoesReais,aparicoesFalsas,bc
0,%,0.000003,0.000086,0.926183
1,&,0.000003,0.000047,0.870162
4,-pe,0.000003,0.000013,0.597284
7,0,0.000016,0.000002,0.777381
13,0000,0.000003,0.000013,0.597284
...,...,...,...,...
19233,😡,0.000003,0.000037,0.837035
19234,🙄🙄🙄,0.000003,0.000073,0.914297
19235,🚨,0.000003,0.000013,0.597284
19236,🤣🤣🤣,0.000003,0.000073,0.914297


In [ ]:
import numpy as np

In [ ]:
dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\2118644562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
C:\Users\CEITELABINFO\AppData\Local\Temp\ipykernel_12024\2118644562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])


In [ ]:
noticia = 'Agora não sei se você viu lá no grupo que eu postei hoje, as pessoas novas que tiraram o título esse ano ou os que transferiram, que foi o meu caso, que eu transferi de Caldas Novas para cá, o título, eles dão um papel só, uma folha de papel compridinho assim, que é para você registrar na hora. Aí lá tem um QR Code que tem escrito Lula 13.'
noticia = noticia.lower()
noticia = noticia.split()
lista = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&']
noticiaFormatada = []

for palavras in noticia:
    for char in lista:
        palavras = palavras.replace(char, '')
    noticiaFormatada.append(palavras)

In [ ]:
dfNoticia = pd.DataFrame(data = noticiaFormatada, columns=['palavra'])
dfNoticia

,palavra
0,agora
1,não
2,sei
3,se
4,você
...,...
65,que
66,tem
67,escrito
68,lula


In [ ]:
dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

In [ ]:
iFalsa = dfNoticia['aparicoesFalsas'].sum()
iReal = dfNoticia['aparicoesReais'].sum()

In [ ]:
print(f'{iFalsa}\t{iReal}')

-98.09150096776183	-116.72186390297516


In [ ]:
if (iFalsa > iReal):
    print("Noticia Falsa!")
else:
    print("Noticia Real!")

Noticia Falsa!
